![image](https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-3-3-8b-instruct` to make simple chat conversation and tool calls

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for Chat models, including the integration of tools and watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.12.


## Learning goal

The purpose of this notebook is to demonstrate how to use Chat models, e.g. `ibm/granite-3-3-8b-instruct` for using tools.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies

In [1]:
%pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1
%pip install -U ibm_watsonx_ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

### Working with projects

First of all, you need to create a project that will be used for your work. If you do not have a project created already, follow the steps below:

- Open IBM Cloud Pak main page
- Click all projects
- Create an empty project
- Copy `project_id` from url and paste it below

**Action**: Assign project ID below

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

#### Create `APIClient` instance

In [5]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials, project_id=project_id)

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


#### Specify chat model

This notebook uses chat model `ibm/granite-3-3-8b-instruct`, which has to be available on your Cloud Pak for Data environment for this notebook to run successfully.  
If this model is not available on your Cloud Pak for Data environment, you can specify any other available chat model.  
You can list available chat models by running the cell below.

In [6]:
if len(client.foundation_models.ChatModels):
    print(*client.foundation_models.ChatModels, sep="\n")
else:
    print(
        "Chat models are missing in this environment. Install chat models to proceed."
    )

ibm/granite-3-3-8b-instruct
meta-llama/llama-3-3-70b-instruct
mistralai/mistral-small-3-1-24b-instruct-2503


In [7]:
model_id = client.foundation_models.ChatModels.GRANITE_3_3_8B_INSTRUCT

### Define the model parameters

You might need to adjust model `parameters` depending on the model you use.

In [8]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-------------------+----------------------------------------+------------------------------+
| PARAMETER         | TYPE                                   | EXAMPLE VALUE                |
+===================+========================================+==============================+
| frequency_penalty | float, NoneType                        | 0.5                          |
+-------------------+----------------------------------------+------------------------------+
| logprobs          | bool, NoneType                         | True                         |
+-------------------+----------------------------------------+------------------------------+
| top_logprobs      | int, NoneType                          | 3                            |
+-------------------+----------------------------------------+------------------------------+
| presence_penalty  | float, NoneType                        | 0.3                          |
+-------------------+---------------------------------------

In [9]:
params = TextChatParameters(temperature=1)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [10]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, credentials=credentials, project_id=project_id, params=params
)

<a id="chat"></a>
## Work with chat messages 

#### Work with a simple chat message

In [11]:
messages = [{"role": "user", "content": "What is 1 + 1"}]

simple_chat_response = model.chat(messages=messages)

In [12]:
print(simple_chat_response["choices"][0]["message"]["content"])

1 + 1 equals 2.


#### Work with a simple chat message using chat_stream

In [13]:
messages = [{"role": "user", "content": "What IBM mainly does?"}]

simple_chat_stream_response = model.chat_stream(messages=messages)

In [14]:
for chunk in simple_chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

IBM integrates technology and expertise, providing business applications, infrastructure, and consulting services to clients globally. They emphasize areas like AI, cloud computing, hybrid computing, and data analytics. Notably, IBM was instrumental in developing the ORDL (Universal Automatic Computing Machine) and later contributed significantly to the personal computer revolution. 

Their product portfolio includes hardware (at one time, IBM was a leading PC manufacturer), software (like their Watson AI and Red Hat division), and infrastructure services. IBM also prioritizes research, operating 19 research facilities worldwide with focuses on vital fields such as semiconductors, nano-tech, quantum computing, blockchain, and the IoT (Internet of Things). 

In essence, IBM serves as both a technology innovator and a service provider, assisting businesses with their digital transformations while continuously pushing the frontiers of technological research.

#### Work with an advanced chat message

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [{"type": "text", "text": "Who won the world series in 2020?"}],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020.",
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": "Where was it played?"}],
    },
]

advanced_chat_response = model.chat(messages=messages)

In [16]:
print(advanced_chat_response["choices"][0]["message"]["content"])

The 2020 World Series was played at the Globe Life Field in Arlington, Texas. This was the first time the World Series was held in a neutral site, with no home-field advantage for the participating teams.


#### Work with chat messages using `tools` and `tool_choice`

In [17]:
def get_current_weather(location: str, unit: str = "celsius") -> dict:
    """
    Get the current weather in a given location.

    Parameters:
    - location (str): The city and state, e.g., "San Francisco, CA".
    - unit (str): The unit for temperature, either "celsius" or "fahrenheit". Defaults to "celsius".

    Returns:
    - dict: A dictionary containing the current weather details.
    """
    weather_data = {
        "location": location,
        "temperature": 20,
        "unit": unit,
        "description": "Partly cloudy",
    }
    return weather_data

Converting raw Python function to correct tool schema

In [18]:
import json
from langchain_ibm.chat_models import convert_to_openai_tool

formatted_tools = [convert_to_openai_tool(get_current_weather)]
print(json.dumps(formatted_tools, indent=4))

[
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location. Parameters:\n- location (str): The city and state, e.g., \"San Francisco, CA\".\n- unit (str): The unit for temperature, either \"celsius\" or \"fahrenheit\". Defaults to \"celsius\".",
            "parameters": {
                "properties": {
                    "location": {
                        "type": "string"
                    },
                    "unit": {
                        "default": "celsius",
                        "type": "string"
                    }
                },
                "required": [
                    "location"
                ],
                "type": "object"
            }
        }
    }
]


In [19]:
messages = [
    {"role": "user", "content": "What's the weather like in Boston today?"},
]

tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}

tool_response = model.chat(
    messages=messages, tools=formatted_tools, tool_choice=tool_choice
)

In [20]:
print(json.dumps(tool_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "content": "",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-bc023d18b56749c69c34e3e8967d2d68",
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "arguments": "{\n  \"location\": \"Boston, MA\",\n  \"unit\": \"fahrenheit\"\n}"
            }
        }
    ]
}


#### Work with chat messages using `tools` and `tool_choice_option`

In [21]:
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers.

    Parameters:
    - a (float): The first number.
    - b (float): The second number.

    Returns:
    - float: The result of a + b.
    """
    return a + b


def multiply_numbers(a: float, b: float) -> float:
    """
    Multiplies two numbers.

    Parameters:
    - a (float): The first number.
    - b (float): The second number.

    Returns:
    - float: The result of a * b.
    """
    return a * b

Converting raw python function to correct tool schema

In [22]:
tool_choice_option = "auto"

formatted_tools = [
    convert_to_openai_tool(tool) for tool in [add_numbers, multiply_numbers]
]

print(json.dumps(formatted_tools, indent=4))

[
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Adds two numbers. Parameters:\n- a (float): The first number.\n- b (float): The second number.",
            "parameters": {
                "properties": {
                    "a": {
                        "type": "number"
                    },
                    "b": {
                        "type": "number"
                    }
                },
                "required": [
                    "a",
                    "b"
                ],
                "type": "object"
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "multiply_numbers",
            "description": "Multiplies two numbers. Parameters:\n- a (float): The first number.\n- b (float): The second number.",
            "parameters": {
                "properties": {
                    "a": {
                        "type": "number"
     

In [23]:
messages = [
    {"role": "user", "content": "What is 5 + 6?"},
]

tools_response = model.chat(
    messages=messages, tools=formatted_tools, tool_choice_option=tool_choice_option
)

print(json.dumps(tools_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-aa024a892309416cad702b6d3bc750ad",
            "type": "function",
            "function": {
                "name": "add_numbers",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


In [24]:
messages = [
    {"role": "user", "content": "What is 5 * 6?"},
]

tools_response_2 = model.chat(
    messages=messages, tools=formatted_tools, tool_choice_option=tool_choice_option
)

print(json.dumps(tools_response_2["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-2326e8dff55c4e80b0474756bc98767a",
            "type": "function",
            "function": {
                "name": "multiply_numbers",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


#### Execute tool calls

We organize the two functions into a dictionary where keys represent the function name, and values are the function.

In [25]:
names_to_functions = {
    "add_numbers": add_numbers,
    "multiply_numbers": multiply_numbers,
}

In [26]:
tool_call = tools_response["choices"][0]["message"]["tool_calls"]
function_name = tool_call[0]["function"]["name"]
function_params = json.loads(tool_call[0]["function"]["arguments"])
print(f"Executing function: `{function_name}`, with parameters: {function_params}")

function_result = names_to_functions[function_name](**function_params)
print(f"Function result: {function_result}")

Executing function: `add_numbers`, with parameters: {'a': 5, 'b': 6}
Function result: 11


In [27]:
tool_call = tools_response_2["choices"][0]["message"]["tool_calls"]
function_name = tool_call[0]["function"]["name"]
function_params = json.loads(tool_call[0]["function"]["arguments"])
print(f"Executing function: `{function_name}`, with parameters: {function_params}")


function_result = names_to_functions[function_name](**function_params)
print(f"Function result: {function_result}")

Executing function: `multiply_numbers`, with parameters: {'a': 5, 'b': 6}
Function result: 30


<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.